## Import

In [1]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab
from bertopic import BERTopic
import pandas as pd

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [100]:
# 공백 정리
import re, unicodedata
from string import whitespace
from pykospacing import Spacing
from nltk import word_tokenize, sent_tokenize

pattern_whitespace = re.compile(f'[{whitespace}]+')

# TBD: newline도 space로 대체하면 괜찮은가?
df['text'] = df.context.str.replace(
    pattern_whitespace, ' '
).map(lambda x: unicodedata.normalize('NFC', x)).str.strip()

#df['subtitle'] = df.description.fillna('').replace(
#    pattern_whitespace, ' '
#).map(lambda x: unicodedata.normalize('NFC', x)).str.strip()

def clean_byline(text):
    # byline
    pattern_email = re.compile(r'[-_0-9a-z]+@[-_0-9a-z]+(?:\.[0-9a-z]+)+', flags=re.IGNORECASE)
    pattern_url = re.compile(r'(?:https?:\/\/)?[-_0-9a-z]+(?:\.[-_0-9a-z]+)+', flags=re.IGNORECASE)
    pattern_others = re.compile(r'\.([^\.]*(?:기자|특파원|교수|작가|대표|논설|고문|주필|부문장|팀장|장관|원장|연구원|이사장|위원|실장|차장|부장|에세이|화백|사설|소장|단장|과장|기획자|큐레이터|저작권|평론가|©|©|ⓒ|\@|\/|=|▶|무단|전재|재배포|금지|\[|\]|\(\))[^\.]*)$')
    result = pattern_email.sub('', text)
    result = pattern_url.sub('', result)

    # 본문 시작 전 꺽쇠로 쌓인 바이라인 제거
    pattern_bracket = re.compile(r'^((?:\[.+\])|(?:【.+】)|(?:<.+>)|(?:◆.+◆)\s)')
    result = pattern_bracket.sub('', result).strip()

    return result


def text_filter(text):
    import re
    from string import whitespace, punctuation
    text = clean_byline(text)
    
    exclude_pattern = re.compile(r'[^\% 0-9a-zA-Zㄱ-ㅣ가-힣.,]+')

    exclusions = exclude_pattern.findall(text)
    result = exclude_pattern.sub(' ', text).strip()
    
    spacing = Spacing()
    kospacing_txt = spacing(result) 
    sentences = sent_tokenize(kospacing_txt)
    return sentences

df['text'] = df.text.map(text_filter)

1/1 [==============================] - 0s 83ms/step


In [109]:
txt = [[(j,t) for j,t in enumerate(v[1])] for v in df['text']]
len(txt)

30

In [110]:
df['text'] = txt

In [111]:
df['text'][4]

[(0, '삼성전자가 갤럭시 버즈 2 프로 소프트웨어 업데이트를 통해 스마트폰 카메라 사용성을 한 단계 더 높였습니다.'),
 (1,
  '삼성전자는 휴대전화로 동영상을 찍을 때 360도 사운드 녹음이 가능하도록 갤럭시 버즈 2 프로와 갤럭시Z 플립4, 폴드 4의 소프트웨어 업데이트를 시행한다고 밝혔습니다.'),
 (2,
  '소프트웨어 업데이트를 통해 버즈 2 프로 좌우에 있는 마이크로 사용자가 듣는 그대로 의 소리를 생생하게 녹음할 수 있다고 삼성전자 측은 설명했습니다.'),
 (3, '당신의 제보가 뉴스가 됩니다 카카오톡 YTN 검색해 채널 추가 전화 02 398 8585 메일')]

In [97]:
df.to_csv('result30_pre.csv')

In [98]:
df = pd.read_csv('result30.csv')
contexts = df['context']

In [99]:
for i in range(len(df)):
    print(i, df['context'][i])

0 = 삼성전자는 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할 수 있도록 갤럭시 버즈2 프로 및 갤럭시 Z 플립4·폴드4의 소프트웨어 업데이트를 13일부터 시행한다고 밝혔다.소프트웨어를 업데이트하면 버즈2 프로 좌우에 있는 마이크가 모두 사용돼 360도에서 들려오는 소리를 플립4·폴드4로 녹음할 수 있다. 녹음에는 차세대 블루투스 오디오 기술 표준인 LE 오디오가 적용됐다.이와 함께 삼성전자는 다음 달 갤럭시 워치5와 워치4 시리즈에 카메라 줌 인, 줌 아웃 기능을 추가한다고 전했다.삼성전자는 "갤럭시 제품 간 연동을 통해 더욱 향상된 카메라 경험을 제공할 것"이라며 "다음 달 진행되는 갤럭시 언팩에서 새로운 갤럭시 스마트폰과 관련된 더 많은 내용을 소개할 것"이라고 말했다
1 [=김진이 기자] 삼성전자는 갤럭시 버즈2 프로와 갤럭시 Z 플립4·폴드4의 소프트웨어(SW) 업데이트를 13일부터 시행한다. 스마트폰 동영상 촬영 시 전방위서 들려오는 소리를 녹음할 수 있게 하기 위해서다. 소프트웨어를 업데이트하면 버즈2 프로 좌우에 있는 마이크가 모두 사용돼 360도에서 들려오는 소리를 플립4·폴드4로 녹음할 수 있다. 녹음에는 차세대 블루투스 오디오 기술 표준인 LE 오디오가 적용됐다. 이와 함께 삼성전자는 다음 달 갤럭시 워치5와 워치4 시리즈에 카메라 줌 인, 줌 아웃 기능을 추가한다. 삼성전자는 스마트폰 동영상 촬영 시 360도 사운드 녹음이 가능하도록 갤럭시 버즈2 프로와 갤럭시 Z 플립4∙폴드4의 소프트웨어 업데이트를 시행한다. 사진=삼성전자 버즈2 프로를 착용하고 '360 오디오 레코딩' 기능을 통해 연결된 Z 플립4·폴드4로 북적거리는 행사장 혹은 자연의 소리가 가득한 숲 한가운데서 동영상을 촬영할 경우 촬영자는 연결된 버즈2 프로를 통해 본인이 듣는 그대로의 생생한 몰입형 사운드를 녹음할 수 있게 된다. 이는 갤럭시 버즈2 프로의 좌우에 위치한 마이크를 모두 사용해 360도 사운드를 포착하기 때문이다. 전문 장비 없이도 누구나 언제 

## Pre-processing

In [4]:
df = pd.read_csv('result100.csv')
df.head()

,Unnamed: 0,title,originallink,link,description,pubDate,context
0,0,"<b>삼성전자</b>, <b>버즈2프로</b> SW <b>업데이트</b>…&quot...",https://www.yna.co.kr/view/AKR2023011303230001...,https://n.news.naver.com/mnews/article/001/001...,<b>삼성전자</b>는 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할...,"Fri, 13 Jan 2023 09:12:00 +0900",= 삼성전자는 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할 수 있도...
1,1,"<b>삼성전자</b>, <b>버즈2프로</b> SW <b>업데이트</b>…스마트폰 ...",http://www.ezyeconomy.com/news/articleView.htm...,http://www.ezyeconomy.com/news/articleView.htm...,<b>삼성전자</b>는 갤럭시 <b>버즈2 프로</b>와 갤럭시 Z 플립4·폴드4의...,"Mon, 16 Jan 2023 10:04:00 +0900",[=김진이 기자] 삼성전자는 갤럭시 버즈2 프로와 갤럭시 Z 플립4·폴드4의 소프트...
2,2,"<b>삼성전자</b>, <b>버즈2프로</b> SW <b>업데이트</b>…&quot...",https://www.joongang.co.kr/article/25133414,https://n.news.naver.com/mnews/article/025/000...,"<b>삼성전자</b>에 따르면, 이날부터 소프트웨어를 <b>업데이트</b>하면 <b...","Fri, 13 Jan 2023 10:46:00 +0900",삼성전자가 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할 수 있도록 ...
3,3,<b>삼성전자 버즈2 프로 업데이트</b>···“360도 소리 녹음 지원”,https://www.khan.co.kr/economy/business/articl...,https://n.news.naver.com/mnews/article/032/000...,<b>삼성전자</b>는 스마트폰으로 동영상 촬영 시 전방위에서 들려오는 소리를 녹음...,"Fri, 13 Jan 2023 11:58:00 +0900",삼성전자는 스마트폰으로 동영상 촬영 시 전방위에서 들려오는 소리를 녹음할 수 있도록...
4,4,"[기업] <b>삼성</b>, 무선이어폰 <b>업데이트</b>로 360도 사운드 녹음 가능",https://www.ytn.co.kr/_ln/0102_202301131647168253,https://n.news.naver.com/mnews/article/052/000...,<b>삼성전자</b>가 갤럭시 <b>버즈2 프로</b> 소프트웨어 <b>업데이트</...,"Fri, 13 Jan 2023 16:47:00 +0900",삼성전자가 갤럭시 버즈2 프로 소프트웨어 업데이트를 통해 스마트폰 카메라 사용성을 ...


In [5]:
descriptions = df['description'].to_list()
descriptions[0]

'<b>삼성전자</b>는 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할 수 있도록 갤럭시 <b>버즈2 프로</b> 및 갤럭시 Z 플립4·폴드4의 소프트웨어 <b>업데이트</b>를 13일부터 시행한다고 밝혔다. 소프트웨어를 <b>업데이트</b>하면... '

In [6]:
len(descriptions[0])

148

In [7]:
preprocessed_descriptions = []

# 빈 문자열이거나 숫자로만 이루어진 줄은 제외
for line in tqdm(descriptions) :
    # if line and not line.replace(' ', '').isdecimal() :
    preprocessed_descriptions.append(line)

len(preprocessed_descriptions)

100%|██████████| 100/100 [00:00<00:00, 394943.88it/s]


100

## BERTopic

In [6]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        # sent = sent[:1000000] # if Error?
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [7]:
custom_tokenizer = CustomTokenizer(Mecab())

In [8]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [9]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 top_n_words=10,
                 # nr_topics=10,
                 calculate_probabilities=True, 
                 verbose=True)

In [10]:
topics, probs = model.fit_transform(preprocessed_descriptions)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2023-01-17 00:51:36,281 - BERTopic - Transformed documents to Embeddings
2023-01-17 00:51:48,694 - BERTopic - Reduced dimensionality
2023-01-17 00:51:48,816 - BERTopic - Clustered reduced embeddings


In [11]:
topic_df = model.get_topic_info()
topic_df

,Topic,Count,Name
0,-1,175,-1_</_네이버_.._인수
1,0,77,0_웹툰_apos_고수_지존
2,1,56,1_최수연_샌프란시스코_대표_레드우드
3,2,53,2_케어_안부_로바_전화
4,3,46,3_페이_접속_장애_발생
5,4,42,4_주가_하락_거래일_상승
6,5,41,5_인수_마크_포시_달러
7,6,37,6_대표_김유원_단독_체제
8,7,35,7_라이브_쇼핑_브랜드_진행
9,8,34,8_북미_패션_최대_마크


In [12]:
model.get_topic(topic=12)

[('무료', 0.08954459941175881),
 ('검색', 0.07564155367717192),
 ('카카오톡', 0.07514973110653907),
 ('이나', 0.060302933704126364),
 ('시장', 0.05122337023765225),
 ('카카오', 0.04820021575391271),
 ('서비스', 0.042227398569085896),
 ('온라인', 0.03892293714772893),
 ('플랫', 0.03723482211547875),
 ('..', 0.0369823426380109)]

In [13]:
src_df = model.get_document_info(preprocessed_descriptions)
src_df.head()

,Document,Topic,Name,Top_n_words,Probability,Representative_document
0,&quot; <b>네이버</b>웹툰을 &apos;아시아의 디즈니’로 만들겠다는 김준...,17,17_웹툰_김준구_엔터_요기,웹툰 - 김준구 - 엔터 - 요기 - 미국 - 연재 - 슈퍼주니어 - 대표 - 세계...,0.305293,False
1,17일 빅데이터 플랫폼 기업 아이지에이웍스의 모바일인덱스가 작년 1월과 12월을 기...,-1,-1_</_네이버_.._인수,</ - 네이버 - .. - 인수 - 마크 - 으로 - 포시 - 플랫 - 카카오 - 에서,0.702552,False
2,"-1위 <b>네이버</b>웹툰, 카카오엔터보다 높은 기업가치 기대감 드러내 글로벌 ...",12,12_무료_검색_카카오톡_이나,무료 - 검색 - 카카오톡 - 이나 - 시장 - 카카오 - 서비스 - 온라인 - 플...,0.064715,False
3,<b>네이버</b>가 개인 간 거래(C2C) 및 온라인 중고거래 기업에 대한 투자를...,11,11_quot_포시_마크_커뮤니티,quot - 포시 - 마크 - 커뮤니티 - 기술력 - 커머스 - 네이버 - </ -...,0.277657,False
4,<b>네이버</b>가 1조6000억원에 산 ‘포시마크’ 마니시 샨드라 CEO “<b...,-1,-1_</_네이버_.._인수,</ - 네이버 - .. - 인수 - 마크 - 으로 - 포시 - 플랫 - 카카오 - 에서,0.309385,False


In [14]:
dst_df = pd.concat([df, src_df['Topic'], src_df['Top_n_words']], axis=1)
dst_df

,Unnamed: 0,title,originallink,link,description,pubDate,Topic,Top_n_words
0,0,美 불모지 뚫은 <b>네이버</b>웹툰 &quot;우리의 경쟁사는 넷플릭스&quot;,http://www.newsis.com/view/?id=NISX20230115_00...,https://n.news.naver.com/mnews/article/003/001...,&quot; <b>네이버</b>웹툰을 &apos;아시아의 디즈니’로 만들겠다는 김준...,"Tue, 17 Jan 2023 09:00:00 +0900",17,웹툰 - 김준구 - 엔터 - 요기 - 미국 - 연재 - 슈퍼주니어 - 대표 - 세계...
1,1,국내 검색앱 사용 감소…<b>네이버</b>-구글 격차는 커져,http://www.wowtv.co.kr/NewsCenter/News/Read?ar...,https://n.news.naver.com/mnews/article/215/000...,17일 빅데이터 플랫폼 기업 아이지에이웍스의 모바일인덱스가 작년 1월과 12월을 기...,"Tue, 17 Jan 2023 09:16:00 +0900",-1,</ - 네이버 - .. - 인수 - 마크 - 으로 - 포시 - 플랫 - 카카오 - 에서
2,2,<b>네이버</b>웹툰 美 증시상장 “계획대로”…포시마크 시너지까지,https://www.ddaily.co.kr/news/article.html?no=...,https://n.news.naver.com/mnews/article/138/000...,"-1위 <b>네이버</b>웹툰, 카카오엔터보다 높은 기업가치 기대감 드러내 글로벌 ...","Tue, 17 Jan 2023 09:13:00 +0900",12,무료 - 검색 - 카카오톡 - 이나 - 시장 - 카카오 - 서비스 - 온라인 - 플...
3,3,"포시마크 인수·왈라팝 투자까지…<b>네이버</b>, C2C에 &apos;베팅&apo...",http://www.inews24.com/view/1558628,https://n.news.naver.com/mnews/article/031/000...,<b>네이버</b>가 개인 간 거래(C2C) 및 온라인 중고거래 기업에 대한 투자를...,"Mon, 16 Jan 2023 16:40:00 +0900",11,quot - 포시 - 마크 - 커뮤니티 - 기술력 - 커머스 - 네이버 - </ -...
4,4,"“<b>네이버</b> 기술로 혁신, 2024년 흑자 만들겠다”",https://www.chosun.com/economy/tech_it/2023/01...,https://n.news.naver.com/mnews/article/023/000...,<b>네이버</b>가 1조6000억원에 산 ‘포시마크’ 마니시 샨드라 CEO “<b...,"Mon, 16 Jan 2023 03:01:00 +0900",-1,</ - 네이버 - .. - 인수 - 마크 - 으로 - 포시 - 플랫 - 카카오 - 에서
...,...,...,...,...,...,...,...,...
995,995,카페멤버 구독 이벤트… `끈끈한` 커뮤니티 키우는 <b>네이버</b>,http://www.dt.co.kr/contents.html?article_no=2...,https://n.news.naver.com/mnews/article/029/000...,<b>네이버</b>가 미래 먹거리로 점찍은 차세대 커뮤니티 서비스 강화에 힘을 쏟고...,"Thu, 05 Jan 2023 14:10:00 +0900",-1,</ - 네이버 - .. - 인수 - 마크 - 으로 - 포시 - 플랫 - 카카오 - 에서
996,996,"<b>네이버</b>·카카오, 초거대 AI의 범용성 높인다",https://www.yna.co.kr/view/AKR2023010312330001...,https://n.news.naver.com/mnews/article/001/001...,경량화로 활용성 확장하거나 오픈 API로 공개 계획 국내 양대 빅테크 <b>네이버<...,"Thu, 05 Jan 2023 07:05:00 +0900",23,ai - 부스트 - 캠프 - 재단 - 커넥트 - 조직 - 기술 - tech - 인공...
997,997,<b>네이버</b>페이 40분 장애…&quot;안정성 강화 중 오류&quot;,http://www.yonhapnewstv.co.kr/MYH2023010502210...,https://n.news.naver.com/mnews/article/422/000...,간편결제 서비스 <b>네이버</b>페이가 오늘(5일) 오후 약 40분간 결제가 정상...,"Thu, 05 Jan 2023 21:24:00 +0900",13,결제 - 주문 - 쇼핑 - 페이지 - 오후 - 실패 - 오류 - 발생 - 처리 - 페이
998,998,"<b>네이버</b>클라우드, 김유원 단독대표 체제로",http://www.aitimes.com/news/articleView.html?i...,http://www.aitimes.com/news/articleView.html?i...,김유원 <b>네이버</b>클라우드 대표 <b>네이버</b>클라우드는 지난 12월30...,"Mon, 02 Jan 2023 17:36:00 +0900",6,대표 - 김유원 - 단독 - 체제 - 클라우드 - 변경 - 이사회 - 공동 - 박원...


In [15]:
dst_df.to_csv('bertopic_result.csv', index=False)

In [16]:
new_df = pd.read_csv('bertopic_result.csv')
new_df.head(1)

,Unnamed: 0,title,originallink,link,description,pubDate,Topic,Top_n_words
0,0,美 불모지 뚫은 <b>네이버</b>웹툰 &quot;우리의 경쟁사는 넷플릭스&quot;,http://www.newsis.com/view/?id=NISX20230115_00...,https://n.news.naver.com/mnews/article/003/001...,&quot; <b>네이버</b>웹툰을 &apos;아시아의 디즈니’로 만들겠다는 김준...,"Tue, 17 Jan 2023 09:00:00 +0900",17,웹툰 - 김준구 - 엔터 - 요기 - 미국 - 연재 - 슈퍼주니어 - 대표 - 세계...


In [17]:
new_df.title[0], new_df.description[0]

('美 불모지 뚫은 <b>네이버</b>웹툰 &quot;우리의 경쟁사는 넷플릭스&quot;',
 '&quot; <b>네이버</b>웹툰을 &apos;아시아의 디즈니’로 만들겠다는 김준구 대표의 꿈이 한 발 더 가까워지고 있다. 웹툰... 공유하고, &quot;<b>네이버</b>웹툰은 압도적인 글로벌 1위 스토리테크 플랫폼을 넘어 전세계 톱티어 엔터테인먼트 기업으로... ')

In [18]:
new_df.Top_n_words[0]

'웹툰 - 김준구 - 엔터 - 요기 - 미국 - 연재 - 슈퍼주니어 - 대표 - 세계 - 떨어지'

In [19]:
topic_0 = new_df[(new_df.Topic == 0)]

topic_0.title.iloc[0], topic_0.description.iloc[0]

('美 진출 <b>네이버</b>웹툰 김준구 대표 &quot;경쟁사는 웹툰 아닌 넷플릭스&quot;',
 '생각해요&quot; <b>네이버</b>웹툰 김준구 대표는 지난 12일(현지시간) 미국 샌프란시스코 힐튼 샌프란시스코 에어포트... <b>네이버</b>웹툰은 2014년 영어 서비스를 시작하고, 2016년 &apos;웹툰 엔터테인먼트&apos;로 미국 시장에 본격 진출했다.... ')

In [20]:
topic_0.Top_n_words.iloc[0]

'웹툰 - apos - 고수 - 지존 - 절대 - 워즈 - 골든디스크 - </ - 네이버 - 무협'